<a href="https://colab.research.google.com/github/KatieMears628/AZLyrics/blob/main/Lyrics_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Katie Mears

# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull.

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it.

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link.

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell.

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.*


# Importing Libraries

In [1]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random
import shutil

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways.

In [2]:
artists = {'morgan wallen': "https://www.azlyrics.com/m/morganwallen.html",
           'pink floyd': "https://www.azlyrics.com/p/pinkfloyd.html"}
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

In [3]:
for artist, artist_page in artists.items():
    try:
        r = requests.get(artist_page, timeout=10)
        r.raise_for_status()
        print(f"Successfully fetched {artist}'s page.")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {artist}'s page: {e}")
    time.sleep(5 + 10 * random.random())

Successfully fetched morgan wallen's page.
Successfully fetched pink floyd's page.


## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).)

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again.

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages.

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know?

A: I believe that the webscraping that we are going to do is allowed. The robots.txt page on www.azlyrics.com has the following instructions:

User-agent: *

Disallow: /lyricsdb/

Disallow: /song/

Allow: /


User-agent: 008

Disallow: /


The first set of instructions specifies that all users are allowed to crawl all parts of the website except for the URLS that start with /lyricsdb/ and /song/. In the URLs provided above for the artists, neither URL contains either of those strings. The second set of instructions blocks User-agent: 008 from crawling any part of the website.


In [4]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    # request the page and sleep
    print(f"Fetching page for {artist}...")
    r = requests.get(artist_page)
    time.sleep(5 + 10 * random.random())

# Parse the artist's page with BeautifulSoup
    soup = BeautifulSoup(r.content, 'html.parser')

# Extract the links to lyrics pages from this page
    for link in soup.find_all('a', href=True):
        url = link['href']

# Check if the URL contains 'lyrics' and starts with '/lyrics/'
        if url.startswith('/lyrics/'):
            full_url = f"https://www.azlyrics.com{url}"
            lyrics_pages[artist].append(full_url)

# Print out the collected lyrics links
for artist, links in lyrics_pages.items():
    print(f"\nLyrics links for {artist}:")
    for link in links:
        print(link)

Fetching page for morgan wallen...
Fetching page for pink floyd...

Lyrics links for morgan wallen:
https://www.azlyrics.com/lyrics/morganwallen/spinyouaround.html
https://www.azlyrics.com/lyrics/morganwallen/sleepwhenweredead.html
https://www.azlyrics.com/lyrics/morganwallen/standalone.html
https://www.azlyrics.com/lyrics/morganwallen/manofthesouth.html
https://www.azlyrics.com/lyrics/morganwallen/yinyanggirl.html
https://www.azlyrics.com/lyrics/morganwallen/goingdown.html
https://www.azlyrics.com/lyrics/morganwallen/scaredtolivewithoutyou.html
https://www.azlyrics.com/lyrics/morganwallen/2ofusalone.html
https://www.azlyrics.com/lyrics/morganwallen/gotitmade.html
https://www.azlyrics.com/lyrics/morganwallen/whatsyourstory.html
https://www.azlyrics.com/lyrics/morganwallen/afterglow.html
https://www.azlyrics.com/lyrics/morganwallen/heavencantakeabackseat.html
https://www.azlyrics.com/lyrics/morganwallen/bonfirejam.html
https://www.azlyrics.com/lyrics/morganwallen/thewayitalk.html
https:

Let's make sure we have enough lyrics pages to scrape.

In [5]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20)

In [6]:
# Let's see how long it's going to take to pull these lyrics
# if we're waiting `5 + 10*random.random()` seconds
for artist, links in lyrics_pages.items() :
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For morgan wallen we have 100.
The full pull will take for this artist will take 0.28 hours.
For pink floyd we have 155.
The full pull will take for this artist will take 0.43 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part.

1. Create an empty folder in our repo called "lyrics".
1. Iterate over the artists in `lyrics_pages`.
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages.
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name.


In [7]:
def generate_filename_from_link(link) :

    if not link :
        return None

    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")

    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")

    # tack on .txt
    name = name + ".txt"

    return(name)


In [8]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") :
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [9]:
url_stub = "https://www.azlyrics.com"
start = time.time()

total_pages = 0

for artist, links in lyrics_pages.items():

    # 1. Build a subfolder for the artist
    artist_folder = os.path.join("lyrics", artist.lower().replace(" ", "_"))
    os.makedirs(artist_folder, exist_ok=True)

    # 2. Iterate over the lyrics pages
    for link in links[:20]:
        print(f"Requesting lyrics for {link}...")

        # 3. Request the lyrics page.
        r = requests.get(link)
        time.sleep(5 + 10 * random.random())

        if r.status_code == 200:
            # 4. Extract the title and lyrics from the page.
            soup = BeautifulSoup(r.content, 'html.parser')
            title = soup.find("h1").text.strip()
            lyrics_div = soup.find(class_="col-xs-12 col-lg-8 text-center")
            lyrics = lyrics_div.get_text(separator="\n").strip() if lyrics_div else "Lyrics not found."

            # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_link`
            #    to generate the filename.
            filename = generate_filename_from_link(link)
            filepath = os.path.join(artist_folder, filename)

            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(f"{title}\n\n{lyrics}")

            total_pages += 1
            print(f"Saved lyrics for {title} in {filepath}.")
        else:
            print(f"Failed to fetch lyrics for {link}. Status code: {r.status_code}")

end = time.time()
print(f"Total pages scraped: {total_pages}")
print(f"Time taken: {end - start:.2f} seconds")

Requesting lyrics for https://www.azlyrics.com/lyrics/morganwallen/spinyouaround.html...
Saved lyrics for "Spin You Around" lyrics in lyrics\morgan_wallen\httpswww_azlyrics_commorganwallen_spinyouaround.txt.
Requesting lyrics for https://www.azlyrics.com/lyrics/morganwallen/sleepwhenweredead.html...
Saved lyrics for "Sleep When We're Dead" lyrics in lyrics\morgan_wallen\httpswww_azlyrics_commorganwallen_sleepwhenweredead.txt.
Requesting lyrics for https://www.azlyrics.com/lyrics/morganwallen/standalone.html...
Saved lyrics for "Stand Alone" lyrics in lyrics\morgan_wallen\httpswww_azlyrics_commorganwallen_standalone.txt.
Requesting lyrics for https://www.azlyrics.com/lyrics/morganwallen/manofthesouth.html...
Saved lyrics for "Man Of The South" lyrics in lyrics\morgan_wallen\httpswww_azlyrics_commorganwallen_manofthesouth.txt.
Requesting lyrics for https://www.azlyrics.com/lyrics/morganwallen/yinyanggirl.html...
Saved lyrics for "Yin Yang Girl" lyrics in lyrics\morgan_wallen\httpswww_azl

In [10]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.13 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [11]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text):
    return re.findall(r'\w+', text.lower())

## Checking Lyrics

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work.

In [12]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders :
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files :
        with open("lyrics/" + artist + "/" + f_name) as infile :
            artist_words.extend(words(infile.read()))


    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For morgan_wallen we have 20 files.
For morgan_wallen we have roughly 14116 words, 1765 are unique.
For pink_floyd we have 20 files.
For pink_floyd we have roughly 8611 words, 1673 are unique.


# Resources

AZLyrics. “AZLyrics - Song Lyrics from a to Z.” Azlyrics.com, 2019, www.azlyrics.com/.

OpenAI. (2024). *ChatGPT* [Large language model]. https://openai.com/chatgpt